In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter
from pathlib import Path
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [4]:
PATH = Path("../input/jigsawtoxic/")
list(PATH.iterdir())

[PosixPath('../input/jigsawtoxic/train.csv'),
 PosixPath('../input/jigsawtoxic/test.csv'),
 PosixPath('../input/jigsawtoxic/embeddings'),
 PosixPath('../input/jigsawtoxic/sample_submission.csv')]

In [5]:
df_train = pd.read_csv(PATH/"train.csv")
df_test = pd.read_csv(PATH/"test.csv")

In [6]:
print("Train shape: ", df_train.shape)
print("Test shape: ", df_test.shape)

Train shape:  (1306122, 3)
Test shape:  (375806, 2)


In [7]:
df_train.head()


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [8]:
df_test.head()


,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [9]:
df_train.target.value_counts()


0    1225312
1      80810
Name: target, dtype: int64

In [10]:
df_train.dtypes

qid              object
question_text    object
target            int64
dtype: object

In [11]:
df_train['question_text'] = df_train['question_text'].astype(str)
df_train['qid'] = df_train['qid'].astype(str)
pip install wordcloud

In [ ]:
from wordcloud import WordCloud 
import plotly.express as px

def nonan(x):
    if type(x) == str:
        return x.replace("\n", "")
    else:
        return ""

text = ' '.join([nonan(abstract) for abstract in df_train["question_text"]])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Common words in comments')

In [ ]:
NOW WE ARE GOING TO DO Tokenization

In [12]:
re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

my_tok = spacy.load('en')
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(sub_br(x))]

In [13]:
text = df_train['question_text'][0]
spacy_tok(text)

['How',
 'did',
 'Quebec',
 'nationalists',
 'see',
 'their',
 'province',
 'as',
 'a',
 'nation',
 'in',
 'the',
 '1960s',
 '?']

**NOw we are going to Compute vocab2index**

In [14]:
counts = Counter()
for q in df_train['question_text']:
    counts.update(spacy_tok(q))

In [15]:
len(counts.keys())


260973

In [16]:
for word in list(counts):
    if counts[word] < 5:
        del counts[word]

In [17]:
len(counts.keys())

57671

In [18]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

DATASET

In [19]:
def encode_sentence(row, vocab2index, N=100, padding_start=True):
    words = spacy_tok(row)
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in words])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

In [20]:
encode_sentence(df_train['question_text'][0], vocab2index, N=100, padding_start=False)


(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       dtype=int32),
 14)

In [20]:
Selection of the max sequece length N according to the 98 percentile


SyntaxError: invalid syntax (<ipython-input-20-1d802cfb1208>, line 1)

In [21]:
x_train_len = np.array([len(q.split()) for q in df_train['question_text']])
x_test_len = np.array([len(q.split()) for q in df_test['question_text']])

In [22]:
x_train_len.max(), x_test_len.max()


(134, 87)

In [23]:
# 99 percentile
N_train = np.percentile(x_train_len, 99)
N_test = np.percentile(x_test_len, 99)

N_train, N_test

(39.0, 39.0)

In [24]:
N = 40


In [ ]:
Split train and  valid the dataset


In [25]:
train, valid = train_test_split(df_train, test_size=0.2)


In [26]:
train.shape, valid.shape


((1044897, 3), (261225, 3))

In [27]:
class Datasets(Dataset):
    def __init__(self, df, vocab2index, is_test=False, N=40, padding_start=True):
        self.question = [encode_sentence(q, vocab2index, N, padding_start) for q in df['question_text']]
        self.is_test = is_test
        if self.is_test:
            self.y = None
        else:
            self.y = list(df["target"])

    def __len__(self):
        return len(self.question)
    
    def __getitem__(self, idx):
        x = self.question[idx]
        if self.is_test:
            return x
        else:
            y = self.y[idx]
            return x, y

In [28]:
train_ds = Datasets(train, vocab2index, N=40, padding_start=False)
valid_ds = Datasets(valid, vocab2index, N=40, padding_start=False)

In [29]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [30]:
x, y = next(iter(train_dl))


In [31]:
x, y = next(iter(train_dl))
len(x)

2

In [32]:
x[0].shape


torch.Size([1000, 40])

In [33]:
train_ds[0]


((array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          123,   92,   19, 1217,   27, 1442,   15], dtype=int32),
  7),
 0)

In [ ]:
LSTM MODEL

In [34]:
class LSTMModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(LSTMModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)  # (batch, seq, feature)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x_emb = self.embeddings(x)
        x_drop = self.dropout(x_emb)
        out_pack, (ht, ct) = self.lstm(x_drop)
        return self.linear(ht[-1])

In [35]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y in train_dl:
            x = torch.tensor(x[0], dtype=torch.long)
            y = y.float()
            y_pred = model(x)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [36]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, y in valid_dl:
        x = torch.tensor(x[0], dtype=torch.long)
        y = y.float().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [37]:
vocab_size = len(words)
print(vocab_size)


57673


In [38]:
model = LSTMModel(vocab_size, 50, 50)


In [ ]:
train_epocs(model, epochs=30, lr=0.01)


train loss 0.115 val loss 0.115 and val accuracy 0.956


In [ ]:
Making predictions by using our trained model

In [39]:
test_ds = Datasets(df_test, vocab2index, is_test=True, N=40, padding_start=False)
test_dl = DataLoader(test_ds, batch_size=1000)

In [40]:
df_submission = pd.read_csv(PATH/"sample_submission.csv")


In [41]:
preds=[]
for x in test_dl:
    x = x[0].long()
    out = model(x)
    pred = (out > 0.0).long()
    preds.append(pred.numpy())

In [42]:
df_submission['prediction'] = np.vstack(preds)


In [43]:
df_submission.head()


,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,1
2,00007756b4a147d2b0b3,1
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0


In [44]:
df_submission.to_csv('submission.csv', index=False)
